In [1]:
#Random Forest algorithm
#replicate findings from Walters et al 2014

In [2]:
#import io, os, sys, types
#from IPython import get_ipython
#from nbformat import read
#from IPython.core.interactiveshell import InteractiveShell

import biom
import sklearn
import pandas as pd

In [3]:
#HMP dataset
text_file = "/home/blooh/ersp/merged_bmi_mapping_final__original_study_HMP__.txt"
biom_file = "/home/blooh/ersp/filtered_otu_table__original_study_HMP__.biom"

In [4]:
m = pd.read_csv(text_file, sep="\t", index_col=0)

In [5]:
m.columns

Index([u'BarcodeSequence', u'LinkerPrimerSequence', u'BMI',
       u'bmi_group_binned', u'bmi_group_coded', u'original_study',
       u'combined_study_bmi_group', u'PCR_PRIMERS', u'TARGET_SUBFRAGMENT',
       u'AGE', u'ELEVATION', u'LONGITUDE', u'COUNTRY', u'SEQUENCING_METH',
       u'SAMPLE_CENTER', u'Description_duplicate', u'ReversePrimer',
       u'COLLECTION_DATE', u'SEX', u'FAMILY_RELATIONSHIP_GG', u'STUDY_CENTER',
       u'EXPERIMENT_CENTER', u'bmi_group_amish', u'RUN_CENTER', u'LATITUDE',
       u'Description'],
      dtype='object')

In [6]:
m.index

Index([u'SRS011271.396367', u'SRS011410.396107', u'SRS011415.394388',
       u'SRS011529.396185', u'SRS011586.393765', u'SRS011621.394454',
       u'SRS012191.395151', u'SRS013177.401537', u'SRS013386.400409',
       u'SRS013543.394407',
       ...
       u'SRS049157.403739', u'SRS049959.401022', u'SRS056620.394943',
       u'SRS057447.396161', u'SRS063068.403097', u'SRS063524.397569',
       u'SRS063827.396534', u'SRS065263.395822', u'SRS065466.395776',
       u'SRS065725.395071'],
      dtype='object', name=u'#SampleID', length=208)

In [7]:
b= biom.load_table(biom_file)

In [8]:
#select only samples that fit LEAN OR OBESE
i = m[m.bmi_group_coded != "Overweight"].index
#print(i)
i_2 = m[m.bmi_group_coded != "Overweight"]
print(i_2)
i_2.loc[i_2.bmi_group_binned == 3, 'bmi_group_binned'] = 0
print(i_2.bmi_group_binned)

                 BarcodeSequence LinkerPrimerSequence  BMI  bmi_group_binned  \
#SampleID                                                                      
SRS011271.396367       TCAGCACGC    ATTACCGCGGCTGCTGG   23                 1   
SRS011410.396107     TCAGAAGCCGC    ATTACCGCGGCTGCTGG   21                 1   
SRS011415.394388     TCAGATCTTAC    ATTACCGCGGCTGCTGG   22                 1   
SRS011529.396185      TCAGTGAAGC    ATTACCGCGGCTGCTGG   24                 1   
SRS011586.393765      TCAGACGCGC    ATTACCGCGGCTGCTGG   23                 1   
SRS011621.394454      TCAGACTCAC    ATTACCGCGGCTGCTGG   19                 1   
SRS012191.395151      TCAGCGCAAC    ATTACCGCGGCTGCTGG   19                 1   
SRS013177.401537     TCAGAGTCGAC   CCGTCAATTCMTTTRAGT   24                 1   
SRS013386.400409    TCAGCCTTCCGC   CCGTCAATTCMTTTRAGT   21                 1   
SRS013543.394407    TCAGAAGAGTTC    ATTACCGCGGCTGCTGG   20                 1   
SRS013687.400715    TCAGAACCTGGC   CCGTC

/home/blooh/.local/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [25]:
writer = pd.ExcelWriter('biom_original_HMP.xlsx', engine='xlsxwriter')

In [26]:
#get biom file
p = pd.DataFrame(b.matrix_data.T.todense().astype(int), index=b.ids(axis="sample"), columns=b.ids(axis="observation"))
#print(p)

In [27]:
# Convert the dataframe to an XlsxWriter Excel object.
p.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [32]:
# load sorted file
sorted_m = pd.read_csv("/home/blooh/ersp/biom_original_HMP.csv", index_col=0)
#print(sorted_m)

CParserError: Error tokenizing data. C error: Expected 8 fields in line 4, saw 16


In [ ]:
data = pd.DataFrame([])
for n in range(0, 143):
    #print(n)
    num = sorted_m.index.get_loc(i[n])
    data = data.append(sorted_m.iloc[[num]])
print(data)

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier


In [ ]:
from sklearn.cross_validation import cross_val_score
iter_range = range(1, 6)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(n_estimators=2000)
    #cross-validation AUC score
    scores = cross_val_score(classifier, data, i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    rForest_scores.append(scores.mean())
    average_accuracy+=scores.mean()

print(rForest_scores)
print(average_accuracy/(5.0))